# Lab - Regularization

## Week 4 Tuesday 11th June

### TASK: Regularized regression
### FUNCTIONS: Ridge, RidgeCV, Lasso, LassoCV
### DOCUMENTATION: http://scikit-learn.org/stable/modules/linear_model.html
### DATA: Crime (n=319 non-null, p=122, type=regression)
### DATA DICTIONARY: http://archive.ics.uci.edu/ml/datasets/Communities+and+Crime

This data set contains data on violent crimes within a community.

In [43]:
########## Prepare data ##########
# read in data, remove categorical features, remove rows with missing values
import pandas as pd
crime = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data', header=None, na_values=['?'])
crime = crime.iloc[:, 5:] # Stripping out descriptive features like county
crime.dropna(inplace=True) # Dropping out missing values.
crime.head()
print(crime.shape)

# define X and y
X = crime.iloc[:, :-1]
y = crime.iloc[:, -1] # last column is goal variable - violent crimes per 100K

# split into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


(319, 123)


In [44]:
# How many columns are in X?
X.shape
# 122 features.

(319, 122)

In [56]:
########## Linear Regression Model Without Regularization ##########
# linear regression
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)
# lm.coef_
list(zip(X.columns.values,lm.coef_))
# What are these numbers?
# Gp - coefficient for every possible feature.

[(5, -3.6618816659782105),
 (6, 0.69812446462463851),
 (7, -0.2619554671596705),
 (8, -0.28527002715028194),
 (9, -0.16474083727537459),
 (10, 0.24697233272421826),
 (11, -1.0929005121936748),
 (12, -0.59685779593661115),
 (13, 1.1120023933804974),
 (14, -0.72196893103299542),
 (15, 4.273465976681706),
 (16, -0.22804026751251311),
 (17, 0.80487576889829482),
 (18, -0.25793473181730497),
 (19, -0.26345802313128558),
 (20, -1.0461695798507957),
 (21, 0.60778419661167216),
 (22, 0.7735525605510527),
 (23, 0.059646802945674343),
 (24, 0.69021592158790068),
 (25, 0.021675943000021583),
 (26, -0.48780294904572419),
 (27, -0.51885840432656016),
 (28, 0.13947881456806871),
 (29, -0.12441794219332974),
 (30, 0.31500382101797386),
 (31, -0.15263373607292369),
 (32, -0.96500392695265746),
 (33, 1.1714216270172748),
 (34, -0.030854669038923768),
 (35, -0.92908554819852307),
 (36, 0.1246545856596646),
 (37, 0.19810450590177639),
 (38, 0.73080482054293505),
 (39, -0.17733729365325046),
 (40, 0.08329

In [55]:
# X.columns
lm.coef_[0]

-3.6618816659782105

In [57]:
# make predictions and evaluate
import numpy as np
from sklearn import metrics
preds = lm.predict(X_test)
print('RMSE (no regularization) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))

RMSE (no regularization) = 0.233813676495


In [58]:
########## Ridge Regression Model ##########
# ridge regression (alpha must be positive, larger means more regularization)
from sklearn.linear_model import Ridge
rreg = Ridge(alpha=0.1, normalize=True)
rreg.fit(X_train, y_train)
rreg.coef_
preds = rreg.predict(X_test)
print('RMSE (Ridge reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))
# Is this model better? Why? 
# RMSE has lowered. Some variables have been removed that don't add information to the model.
# If alpha is zero, this is equivalent to performing linear regression.

RMSE (Ridge reg.) = 0.164279068049


In [59]:
# use RidgeCV to select best alpha
# Very common procedure - ridge regression cross validation.
from sklearn.linear_model import RidgeCV
alpha_range = 10.**np.arange(-2, 3)
rregcv = RidgeCV(normalize=True, scoring='neg_mean_squared_error', alphas=alpha_range)
rregcv.fit(X_train, y_train)

# Print the optimal value of Alpha for Ridge Regression
print('Optimal Alpha Value: ', rregcv.alpha_)

# Print the RMSE for the ridge regression model
preds = rregcv.predict(X_test)
print ('RMSE (Ridge CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))
# What is the range of alpha values we are searching over?

Optimal Alpha Value:  1.0
RMSE (Ridge CV reg.) = 0.163129782343


In [60]:
########## Lasso Regression Model ##########
# lasso (alpha must be positive, larger means more regularization)
from sklearn.linear_model import Lasso
las = Lasso(alpha=0.001, normalize=True)
las.fit(X_train, y_train)
las.coef_
preds = las.predict(X_test)
print('RMSE (Lasso reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))
# GP - commentary - RMSE is 0.2 at alpha 0.01, so try 0.001 to get a better error.

RMSE (Lasso reg.) = 0.160039024044


In [61]:
# try a smaller alpha
las = Lasso(alpha=0.0001, normalize=True)
las.fit(X_train, y_train)
las.coef_
preds = las.predict(X_test)
print('RMSE (Lasso reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))

# GP - Search range and step size is important in feature selection...

RMSE (Lasso reg.) = 0.164502413721


In [62]:
# use LassoCV to select best alpha (tries 100 alphas by default)
from sklearn.linear_model import LassoCV
alpha_range = 10.**np.arange(-5, 3)
lascv = LassoCV(normalize=True, alphas=alpha_range)

# You can screw around with alpha_range - default range bottoms at 0.01.
lascv.fit(X_train, y_train)
print('Optimal Alpha Value: ',lascv.alpha_)
lascv.coef_
preds = lascv.predict(X_test)
print('RMSE (Lasso CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))

Optimal Alpha Value:  0.001
RMSE (Lasso CV reg.) = 0.160039024044


/Applications/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


### Task 1: Carry out Elastic net regularised regression

### Lookup [Elastic Net](http://scikit-learn.org/stable/modules/linear_model.html#elastic-net) and complete the following.



1. What is elastic net?
2. How does it work?
3. Run elastic net on the above dataset

In [15]:
'''
Elastic Net is a great tool to leverage in real life

1. Elastic Net is another method of regularization. 
It combines some of the advantages of ridge regression, 
along with other advantages from lasso (i.e. few features
are non zero). It's useful in data sets where multiple features are correlated
with each other, and where there are more features than data points.
2) ElasticNet works by introducing two regularizers - L1 and L2.
This adds a quadratic element to the penalty.
This is specified using a parameter called the l1_ratio (rho)
3) 

'''

'\nElastic Net is another great model to leverage in real life\nRESEARCH\n1. Another method of regularization. It combines some of the advantages of\n\n'

In [16]:
# Setup the elastic net model
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=0.1, l1_ratio=0.7)
enet.fit(X_train,y_train)
preds2 = enet.predict(X_test)
print('RMSE (enet reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds2)))




RMSE (enet reg.) = 0.242253664949


In [34]:
# Setting up a CV version of the elastic net model
from sklearn.linear_model import ElasticNetCV
# Specifying parameter ranges
alpha_rangeEN = 10.**np.arange(-3, 3)
# l1_ratio_rangeEN = 10.**np.arange(-6,1) 
l1_ratio_rangeEN = [.1, .5, .7, .9, .95, .99, 1] # provides a better result.
enetcv = ElasticNetCV(alphas=alpha_rangeEN,l1_ratio=l1_ratio_rangeEN)
print(enetcv.get_params())
enetcv.fit(X_train,y_train)
print('Optimal Alpha Value: ',enetcv.alpha_)
print('Optimal l1_ratio Value: ',enetcv.l1_ratio_)
preds3 = enetcv.predict(X_test)
print('RMSE (enetcv reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds3)))

{'alphas': array([  1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02]), 'copy_X': True, 'cv': None, 'eps': 0.001, 'fit_intercept': True, 'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], 'max_iter': 1000, 'n_alphas': 100, 'n_jobs': 1, 'normalize': False, 'positive': False, 'precompute': 'auto', 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'verbose': 0}
Optimal Alpha Value:  0.01
Optimal l1_ratio Value:  0.5
RMSE (enetcv reg.) = 0.160399788864


### Task 2: Carry out Regularised Regression

1. Run all three forms of regularised regression on the Boston Housing DataSet
2. What do the coefficients mean?
3. What would you advise someone living in Boston to try and raise the value of their home?

In [42]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston


# GP - importing the data

boston = load_boston()
scaler = StandardScaler()
X = scaler.fit_transform(boston["data"]) # 13 features, 506 data points
Y = boston["target"]
names = boston["feature_names"]

# GP - Lasso model

lasso = Lasso(alpha=.3)
lasso.fit(X, Y)

# GP - LassoCV
# from sklearn.linear_model import LassoCV
# alpha_range = 10.**np.arange(-5, 3)
# lascv = LassoCV(normalize=True, alphas=alpha_range)
# lascv.fit(X, Y)
# print('Optimal Alpha Value: ',lascv.alpha_)

# GP - Ridge

from sklearn.linear_model import Ridge
rreg = Ridge(alpha=0.3, normalize=True)
rreg.fit(X, Y)

# GP - ElasticNet

enet = ElasticNet(alpha=0.3, l1_ratio=0.1)
enet.fit(X_train,y_train)

#A helper method for pretty-printing linear models
def pretty_print_linear(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)
#Print all the model results

print("Lasso model: ", pretty_print_linear(lasso.coef_, names, sort = True))
print("Ridge model: ", pretty_print_linear(rreg.coef_, names, sort = True))
print("ElasticNet model: ", pretty_print_linear(enet.coef_, names, sort = True))

Lasso model:  -3.707 * LSTAT + 2.992 * RM + -1.757 * PTRATIO + -1.081 * DIS + -0.7 * NOX + 0.631 * B + 0.54 * CHAS + -0.236 * CRIM + 0.081 * ZN + -0.0 * INDUS + -0.0 * AGE + 0.0 * RAD + -0.0 * TAX
Ridge model:  -2.723 * LSTAT + 2.68 * RM + -1.556 * PTRATIO + -1.368 * DIS + -0.752 * NOX + 0.743 * B + 0.726 * CHAS + -0.606 * CRIM + -0.528 * TAX + 0.516 * ZN + 0.462 * RAD + -0.443 * INDUS + -0.197 * AGE
ElasticNet model:  0.049 * INDUS + -0.042 * CHAS + 0.0 * CRIM + 0.0 * ZN + -0.0 * NOX + 0.0 * RM + 0.0 * AGE + 0.0 * DIS + 0.0 * RAD + -0.0 * TAX + 0.0 * PTRATIO + 0.0 * B + -0.0 * LSTAT


/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:39: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [ ]:
'''
2)
Definitions of features: https://archive.ics.uci.edu/ml/datasets/housing

'''

In [ ]:
'''
3)
Recommendations
Can't change some things like LSTAT - got to think of levers
Can build more rooms.
Can chose post codes with rich neighbours, high pupil to teacher ratio.

'''